In [ ]:
# Bitcoin Project - PQT
import os
import csv
import requests
import pandas as pd
import tweepy  
import praw
import kaggle as kg
import nltk
import time
from nltk.sentiment import SentimentIntensityAnalyzer
from pytrends.request import TrendReq

In [ ]:
# Set up environment variables
    # Kaggle
KAGGLE_USERNAME = ""
KAGGLE_KEY = ""
    # Reddit
    
YOUR_CLIENT_ID = ""
YOUR_CLIENT_SERET = ""



# Kaggle Credentials
os.environ['KAGGLE_USERNAME'] = '{KAGGLE_USERNAME}'
os.environ['KAGGLE_KEY'] = '{KAGGLE_KEY}'
kg.api.authenticate()

# Download VADER Sentiment Analyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Reddit Credentials
reddit = praw.Reddit(
    client_id="YOUR_CLIENT_ID",
    client_secret="YOUR_CLIENT_SECRET",
    user_agent="YOUR_USER_AGENT"
)

#Twitter Credentials
client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN)

In [ ]:
## Dataset 1: Bitcoin Historical Data
'''Gets min/min bitcoin data since 2012'''

data_1 = ""
df_1 = "onzip"
kg.api.dataset_download_files(dataset = data_1, path=df_1, unzip=True)
df_1 = pd.read_csv("on.zip/btcusd_1-min_data.csv")

## Alternativly you can download it directly as a csv:
''' 
data_1 = "mczielinski/bitcoin-historical-data"
file_name = "btcusd_1-min_data.csv"
kaggle.api.dataset_download_file(dataset=data_1, file_name=file_name, path="./")
df_1 = pd.read_csv(file_name)
'''


Dataset URL: https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data


FileNotFoundError: [Errno 2] No such file or directory: 'on.zip/btcusd_1-min_data.csv'

In [ ]:
## Hint W2:
# Load the dataset
df_test = pd.read_csv("btcusd_1-min_data.csv", parse_dates=["timestamp"])

# Convert timestamp to datetime
df_test["timestamp"] = pd.to_datetime(df_1["timestamp"], unit="s")

# Keep only the last 1 year of data
df_test = df_test[df_test["timestamp"] >= pd.Timestamp.now() - pd.DateOffset(years=1)]

# Save the filtered dataset
df_test.to_csv("bitcoin_last_year.csv", index=False)
print("Dataset reduced to last 1 year and saved as 'bitcoin_last_year.csv'")

In [ ]:
## Dataset 2: Bitcoin/Eth Prices
'''
    Gets day/day bitcoin data from start-Present
    *** Note there are some other useful datasets here 
    Explore!
'''
data_2 = "kapturovalexander/bitcoin-and-ethereum-prices-from-start-to-2023"
df_2 = "on_1.zip"
kg.api.dataset_download_files(dataset = data_2,path=df_2,unzip = True)
''' 
    There are 10 files in the zip: df_2 is currently 
    set to Eth/USD for the month of January, you can 
    change that by replacing on_1.zip/{filename.csv}
    with the name of the file.
'''
df_2 = pd.read_csv("on_1.zip/ETH-USD (01-05.2024).csv")


In [ ]:
## Dataset 3: Bitcoin F&G index
'''Uses live data to get the fear and greed index of bitcoin'''
## get the data (json)
def get_fng_data():
    url = "https://api.alternative.me/fng/"
    params = {"limit": 0, "format": "json", "date_format": "world"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()["data"]
    else:
        print(response.status_code)
        return None
## json->csv
def convert_csv(data, filename):
    if not data:
        print("No data")
        return
    keys = data[0].keys()
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)

## Call functions
## df_3 = get_fng_data()
## convert_csv(df_3,"fear_greed_index.csv")

In [ ]:
## Reddit setup
auth = requests.auth.HTTPBasicAuth(YOUR_CLIENT_ID,YOUR_CLIENT_SERET)
with open('pw.txt','r') as f:
    pw = f.read()
data = {
    'grant_type': 'password',
    'username': 'princeton-pqt',
    'password': 'Charfyee3!123'
}
headers = {'User-Agent': 'MyAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',auth=auth, data=data, headers=headers)
## res.json()
TOKEN = res.json()['access_token']
headers['Authorization'] = f'bearer {TOKEN}'


In [ ]:
## Dataset 5: Reddit Scraping
res = requests.get('https://oauth.reddit.com/r/Bitcoin/hot',headers=headers ) #/new or params = {'limit':100})
res.json()
for post in res.json()['data']['children'][:1]:
   print(post['data'])


post['data'].keys()




In [ ]:
df5 = []
# Loop through the JSON response and collect relevant fields
for post in res.json()['data']['children']:
    df5.append({  # Collect data correctly
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'ups': post['data']['ups'],
        'downs':post['data']['downs']
    })

# Create DataFrame from collected data
df_5 = pd.DataFrame(df5)  # Now it works!

# Save to CSV
df_5.to_csv('reddit_posts.csv', index=False)

In [ ]:
## Dataset 6: Google Search Trends
'''Maps google search trends with keyword 'Bitcoin' '''
def fetch_google(keyword, timeframe, max_retries=3):
    """Fetch Google Search Trends for a given keyword and timeframe, and save to a CSV file with additional insights."""
    
    retry_count = 0  # Track retries

    while retry_count < max_retries:
        try:
            print(f"Fetching Google Trends data for '{keyword}' with timeframe '{timeframe}' (Attempt {retry_count + 1})")

            # Initialize Pytrends with a User-Agent to bypass blocks
            pytrends = TrendReq(hl="en-US", tz=360, requests_args={'headers': {'User-Agent': 'Mozilla/5.0'}})

            # Build the search payload
            pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo="US", gprop="")

            # Fetch the interest over time
            trends_data = pytrends.interest_over_time()

            # Check if data is returned
            if trends_data.empty:
                print("No data returned. Check if the keyword and timeframe are valid.")
                return

            # Remove the "isPartial" column if it exists
            if "isPartial" in trends_data.columns:
                trends_data.drop(columns=["isPartial"], inplace=True)

            # 🔹 Add useful columns
            trends_data["Rolling_Avg"] = trends_data[keyword].rolling(window=7, min_periods=1).mean()  # 7-day moving avg
            trends_data["Normalized_Score"] = (trends_data[keyword] - trends_data[keyword].min()) / \
                                              (trends_data[keyword].max() - trends_data[keyword].min()) * 100  # Scale 0-100

            # Generate timestamped filename
            timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"google_trends_{keyword.lower()}_{timestamp}.csv"
            filepath = os.path.join(os.getcwd(), filename)

            # Save to CSV
            trends_data.to_csv(filepath)
            print(f"Data successfully saved to '{filepath}'")
            return  # Exit function on success

        except Exception as e:
            print(f"Error fetching Google Trends data: {e}")
            retry_count += 1
            wait_time = 5 * (2 ** retry_count)  # Exponential backoff
            print(f"Retrying in {wait_time} seconds...")
            time.sleep(wait_time)

# Fetch Google Trends data for Bitcoin over the last month
fetch_google("Bitcoin", "today 1-m")  # Last 30 days
